## Fasttext
- word2vec은 사전에 존재하지 않는 단어를 잡아내지 못하는 단점이 있다.
- Fasttext를 통해 이러한 OOV 문제를 해결하며 준수한 성능까지 유도할 수 있을지 알아보자.

## Import

In [1]:
import pandas as pd
import numpy as np

from konlpy.tag import Mecab
from gensim.models import FastText

### Datas

In [2]:
df1 = pd.read_csv("data/maeili.csv", encoding='utf-8')
df2 = pd.read_csv("data/baby.csv", encoding='utf-8')
df3 = pd.read_csv("data/toys.csv", encoding='utf-8')
df4 = pd.read_csv("data/seoulchildcare.csv", encoding='utf-8')
df5 = pd.read_csv("data/babynews.csv", encoding='utf-8')
df6 = pd.read_csv("data/ildongfoodis.csv", encoding='utf-8')
df7 = pd.read_csv("data/momsmagazine.csv", encoding='utf-8')

#### maeili(매일아이)

In [3]:
df1.tail()

,categories,title,text,postlink
1533,임신,'웃음요법' 태교,임신 중에는 스트레스를 받지 않고 몸을 편안하게 하는 것만큼 좋은 태교가 없습니다 ...,https://www.maeili.com/cms/contents/contentsVi...
1534,임신,기형 검사와 보조 검사,모든 임신부는 나이에 상관없이 7백 명에 한 명꼴로 정신박약을 초래하는 다운 증후군...,https://www.maeili.com/cms/contents/contentsVi...
1535,임신,임신 시기별 검사 종류와 횟수,첫방문시 혈액검사 소변검사 혈액형 검사 풍진항체 검사 8 18주 초음파검사 필요한...,https://www.maeili.com/cms/contents/contentsVi...
1536,임신,초음파 검사의 임상적 유용성,초기의 정상적인 자궁내 임신의 확인 임신낭과 태아의 성장 여부와 크기 측정 임신주...,https://www.maeili.com/cms/contents/contentsVi...
1537,임신,산전 유전 상담이란,유전 질환 염색체 이상도 말초 혈액의 염색체 검사로서 진단이 가능하나 산과 영역에서...,https://www.maeili.com/cms/contents/contentsVi...


#### momtalk(육아정보)

In [4]:
df2.tail()

,idx,text
695,852,유아용품 유통기한 바로알기 기저귀 화장품 기저귀 화장품에도 유통기한이 있다구 두고...
696,928,우리 아이 쑥쑥 클 수 있게 전지현 크림으로 하는 베이비 마사지법 서 알려드려요 ...
697,971,이유식을 시작하는 맘들을 위해 이유식 준비물 요점 정리 이유식을 시작하는 맘들을 ...
698,843,휴일에 문 여는 약국이 궁금하다면 휴일지킴이약국 검색법 하필 쉬는 날 응급약이 필...
699,842,우리 아이가 아파요 응급실에 가야할까요 소아과 전문의 김연호 선생님과 함께하는 우...


#### momtalk(유아용품)

In [5]:
df3.tail()

,idx,text
482,622,우리아이 치발기 치아발육기 선택 꿀팁 잇몸이 근질근질우리아이 치아발육기 선택팁 아...
483,640,쏘서 점퍼루 보행기 우리아이 선택은 쏘서 점퍼루 보행기 우리아이 선택은 아이를 낳...
484,607,신생아부터 유아까지 아기목욕용품 및 아기 목욕시키기 신생아부터 유아까지아기 목욕용...
485,567,아기용품 어떤걸로 골라야 할까 아기용품 고르기 오늘은 아기용품고르기에 대해 함께 ...
486,604,장난감수납장 이거좋아요 이번주 베스트글 주인공은 소영님 입니다 요즘 장난감수납장에...


#### 서울육아종합지원센터 육아전문칼럼

In [6]:
df4.tail()

,Unnamed: 0,text
34,34,주제 영유아의 선행학습 진짜 도움될까 부제 당신의 잘못이 아닙니다 다만 저는 예전...
35,35,좋은 아빠 지금 하지 않으면 늦다 결혼을 하고 아이를 키우면서 가장 힘든 점은 욕...
36,36,놀이의 뇌과학적 가치 요즘 아이들은 밖에서 같이 놀 기회가 적어졌다 누군가와 놀아...
37,37,영유아기 자녀와 안정 애착 형성하기 영유아기 때 부모와 맺은 관계 경험은 평생을 ...
38,38,아이를 숲으로 데려갈까요 4차 산업혁명시대를 맞이하여 더 많이 놀게 하고 더 많이...


#### 육아신문 베이비뉴스

In [7]:
df5.tail()

,titles,text,postlinks
995,여름만 되면 설사를 달고 살고 배앓이를 하네요,최근 갑자기 덥고 습해진 여름 날씨로 인해 갑자기 설사를 하거나 배앓이를 호소하는 ...,https://www.ibabynews.com/news/articleView.htm...
996,아기가 잘 때 배를 드러내고 벽에 붙어서 자요,7월부터는 일중 최저기온이 20도 이상으로 높아지면서 어른들도 잘 때 더위를 타고 ...,https://www.ibabynews.com/news/articleView.htm...
997,"기다리는 행복, 아가야",얼마 전 필자를 찾아온 그녀는 청각·시각적으로 장애가 있는 난임 여성입니다 그녀는 ...,https://www.ibabynews.com/news/articleView.htm...
998,"심심하게, 아이를 더 '심심하게' 하라!",지난 20여 년 동안 아이를 키워오면서 가장 많이 들었던 말은 다름 아닌 아빠 심심...,https://www.ibabynews.com/news/articleView.htm...
999,클레이로 무당벌레 장난감 만들기,요즘은 아이들이 좋아하는 초콜릿도 아이들의 호기심을 자극하기 위해예쁘고 귀여운 모양...,https://www.ibabynews.com/news/articleView.htm...


#### 일동후디스

In [8]:
df6.tail()

,titles,text,postlinks
1640,육아에 필요한 법률상담,고귀한 탯줄법률상담5살인 저희 아이는 심각한 고등어 알러지가 있습니다 그래서 집에...,http://foodis.com/web/www/baby_column_02;jsess...
1641,이럴 땐 이렇게 하세요,아기를 가진 엄마들의 궁금증현재 두 아기의 엄마이기도 한종로구 의 김현아 선생님께서...,http://foodis.com/web/www/baby_column_02;jsess...
1642,"키 작은 아이, 남들만큼 클 수 없을까요?",키 작은 아이 남들만큼 클 수 없을까요 먹거리가 귀한 시절에 부모는 아이가 단지...,http://foodis.com/web/www/baby_column_02;jsess...
1643,스킨쉽을 통해 엄마의 사랑을 전해주세요.,마사지를 시작하기 전에 1 항상 시작신호를 보내고 아기의 의사를 존중해야한다 ...,http://foodis.com/web/www/baby_column_02;jsess...
1644,전문의에게 배우는 초보 엄마 아기 도우미,초보 엄마 도우미전문의에게 배우는 초보 엄마 아기 도우미우리 아기 응급 처치법 아...,http://foodis.com/web/www/baby_column_02;jsess...


#### 맘스매거진

In [9]:
df7.tail()

,titles,text,postlink
387,‘믿지 마세요.’ 육아 속설들,육아에 대한 말과 정보들은 많지만 근거 없는 육아 속설에 의지해 아이를 돌보면 큰 ...,http://momsmagazine.co.kr/%eb%af%bf%ec%a7%80-%...
388,신생아가 있는 집 에어컨 사용법,신생아는 땀구멍이 열리지 않아 땀을 흘리지 않는다는 이유로 더위를 타지 않을 거라고...,http://momsmagazine.co.kr/%ec%8b%a0%ec%83%9d%e...
389,초등학교 입학 전 읽기 능력을 키우는 법,초등학교 입학을 앞둔 있는 엄마들은 내 아이가 한글을 읽고 쓰는 데 어려움이 없지 ...,http://momsmagazine.co.kr/%ec%b4%88%eb%93%b1%e...
390,아는 만큼 혜택 보는 2015 출산 지원 제도,2015년 정부는 임신과 출산을 위한 지원제도를 마련했다 임산부에게 경제적 심리적 ...,http://momsmagazine.co.kr/%ec%95%84%eb%8a%94-%...
391,우리 아기 여름 땀띠 똑똑하게 관리하기,덥고 습한 여름은 민감한 아기 피부에 트러블이 생기기 쉽다 영유아는 어른보다 땀샘의...,http://momsmagazine.co.kr/%ec%9a%b0%eb%a6%ac-%...


### Concatenate Dataframe

In [10]:
df = pd.concat([df1.text, df2.text, df3.text, df4.text, df5.text, df6.text, df7.text])
df = pd.DataFrame(df).reset_index(drop=True)
df.tail()

,text
5796,육아에 대한 말과 정보들은 많지만 근거 없는 육아 속설에 의지해 아이를 돌보면 큰 ...
5797,신생아는 땀구멍이 열리지 않아 땀을 흘리지 않는다는 이유로 더위를 타지 않을 거라고...
5798,초등학교 입학을 앞둔 있는 엄마들은 내 아이가 한글을 읽고 쓰는 데 어려움이 없지 ...
5799,2015년 정부는 임신과 출산을 위한 지원제도를 마련했다 임산부에게 경제적 심리적 ...
5800,덥고 습한 여름은 민감한 아기 피부에 트러블이 생기기 쉽다 영유아는 어른보다 땀샘의...


### Remove some contents

In [11]:
df = df.drop(index=[952, 955, 956, 960, 962, 964, 1165, 1240, 1322])
df = pd.DataFrame(df).reset_index(drop=True)
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5787 entries, 0 to 5791
Data columns (total 1 columns):
text    5787 non-null object
dtypes: object(1)
memory usage: 90.4+ KB


### FastText 학습에 적합한 말뭉치로 정제하는 과정
- 한글을 자음, 모음 단위로 분해
- 단어 단위로 구분된 ndarray 자료구조로 정제

In [12]:
import re
from soynlp.hangle import decompose, compose

def remove_doublespace(s):
    doublespace_pattern = re.compile('\s+')
    return doublespace_pattern.sub(' ', s).strip()

def encode(s):
    def process(c):
        if c == ' ':
            return c
        jamo = decompose(c)
        # 'a' or 모음 or 자음
        if (jamo is None) or (jamo[0] == ' ') or (jamo[1] == ' '):
            return ' '
        base = jamo[0]+jamo[1]
        if jamo[2] == ' ':
            return base + '-'
        return base + jamo[2]

    s = ''.join(process(c) for c in s)
    return remove_doublespace(s).strip()

#     s = [process(c) for c in s]
#     return s

def decode(s):
    def process(t):
        assert len(t) % 3 == 0
        t_ = t.replace('-', ' ')
        chars = [tuple(t_[3*i:3*(i+1)]) for i in range(len(t_)//3)]
        recovered = [compose(*char) for char in chars]
        recovered = ''.join(recovered)
        return recovered

    return ' '.join(process(t) for t in s.split())

#### Test

In [13]:
s = '나의 야근을 알까? 흐헤헤 흐헤헤 아이고난1 아이고난2'
print(encode(s))
print(decode(encode(s)))

ㄴㅏ-ㅇㅢ- ㅇㅑ-ㄱㅡㄴㅇㅡㄹ ㅇㅏㄹㄲㅏ- ㅎㅡ-ㅎㅔ-ㅎㅔ- ㅎㅡ-ㅎㅔ-ㅎㅔ- ㅇㅏ-ㅇㅣ-ㄱㅗ-ㄴㅏㄴ ㅇㅏ-ㅇㅣ-ㄱㅗ-ㄴㅏㄴ
나의 야근을 알까 흐헤헤 흐헤헤 아이고난 아이고난


In [14]:
df['encoded'] = df['text'].apply(encode)
df

,text,encoded
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...,ㅅㅏㄹ ㄴㅏㅁㅇㅏ- ㅇㅣㅂㄴㅣ-ㄷㅏ- ㅎㅚ-ㅅㅏ- ㅂㅗㄱㅈㅣㄱ ㅎㅜ- ㅅㅏㄹㅂㅜ-ㅌ...
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...,ㅇㅛ-ㅈㅡㅁ ㄷㅡㄹㅇㅓ- ㅇㅏ-ㅃㅏ-ㄱㅏ- ㅇㅏ-ㅇㅣ-ㅇㅔ-ㄱㅔ- ㄷㅏ-ㄱㅏ-ㄱㅏ-...
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...,ㄷㅜㄹㅉㅐ-ㄱㅏ- ㅌㅐ-ㅇㅓ-ㄴㅏ-ㅁㅕㄴ ㅊㅓㅅㅉㅐ-ㄱㅏ- ㅎㅣㅁㄷㅡㄹ ㄱㅓ-ㄹㅏ-...
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...,ㄱㅐ-ㅇㅝㄹ ㄴㅏㅁㅇㅏ-ㄱㅏ- ㅇㅣㅆㄱㅗ- ㅇㅣㅁㅅㅣㄴ ㄱㅐ-ㅇㅝㄹㅊㅏ-ㄹㅗ- ㄷㅜ...
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...,ㄱㅐ-ㅇㅝㄹ ㄴㅏㅁㅈㅏ- ㅇㅏ-ㄱㅣ-ㅇㅣㄴㄷㅔ- ㅅㅣㅁㅎㅏ-ㄱㅔ- ㄲㅐ-ㅁㅜ-ㄴㅡㄴ...
...,...,...
5787,육아에 대한 말과 정보들은 많지만 근거 없는 육아 속설에 의지해 아이를 돌보면 큰 ...,ㅇㅠㄱㅇㅏ-ㅇㅔ- ㄷㅐ-ㅎㅏㄴ ㅁㅏㄹㄱㅘ- ㅈㅓㅇㅂㅗ-ㄷㅡㄹㅇㅡㄴ ㅁㅏㄶㅈㅣ-ㅁㅏㄴ...
5788,신생아는 땀구멍이 열리지 않아 땀을 흘리지 않는다는 이유로 더위를 타지 않을 거라고...,ㅅㅣㄴㅅㅐㅇㅇㅏ-ㄴㅡㄴ ㄸㅏㅁㄱㅜ-ㅁㅓㅇㅇㅣ- ㅇㅕㄹㄹㅣ-ㅈㅣ- ㅇㅏㄶㅇㅏ- ㄸㅏㅁ...
5789,초등학교 입학을 앞둔 있는 엄마들은 내 아이가 한글을 읽고 쓰는 데 어려움이 없지 ...,ㅊㅗ-ㄷㅡㅇㅎㅏㄱㄱㅛ- ㅇㅣㅂㅎㅏㄱㅇㅡㄹ ㅇㅏㅍㄷㅜㄴ ㅇㅣㅆㄴㅡㄴ ㅇㅓㅁㅁㅏ-ㄷㅡㄹ...
5790,2015년 정부는 임신과 출산을 위한 지원제도를 마련했다 임산부에게 경제적 심리적 ...,ㄴㅕㄴ ㅈㅓㅇㅂㅜ-ㄴㅡㄴ ㅇㅣㅁㅅㅣㄴㄱㅘ- ㅊㅜㄹㅅㅏㄴㅇㅡㄹ ㅇㅟ-ㅎㅏㄴ ㅈㅣ-ㅇㅝ...


In [15]:
encoded_corpus = np.array(df.encoded)
type(encoded_corpus), len(encoded_corpus)

(numpy.ndarray, 5787)

### Train

In [16]:
%%time
class FastTextCorpus:
    def __init__(self, corpus):
        self.corpus = corpus
        self.n_iter = 0
    def __iter__(self):
        for i, doc in enumerate(self.corpus):
            if i % 500 == 0 and i > 0:
                print('\riter = {}, sents = {} ...'.format(self.n_iter, i), end='')
            yield doc.split()
        self.n_iter += 1
        print('\riter = {}, docs = {} done{}'.format(self.n_iter, i, ' '*20))
        
corpus = FastTextCorpus(encoded_corpus)
fasttext_model = FastText(
    corpus,
    size=100,
    seed=191212,
    workers=6,
    window=3,
    min_count=6,
    min_n=3,
    max_n=6,
)

iter = 1, docs = 5786 done                    
iter = 2, docs = 5786 done                    
iter = 3, docs = 5786 done                    
iter = 4, docs = 5786 done                    
iter = 5, docs = 5786 done                    
iter = 6, docs = 5786 done                    
CPU times: user 1min 30s, sys: 646 ms, total: 1min 31s
Wall time: 26.1 s


In [17]:
def most_similar(query, topn=10):
    query_ = encode(query)
    similars = fasttext_model.wv.most_similar(query_, topn=topn)
    similars = [(decode(word), sim) for word, sim in similars]
    return similars

In [18]:
most_similar("여자아이")

[('남자아이', 0.9715147018432617),
 ('경희아이', 0.9466871023178101),
 ('아이돌이', 0.9300433397293091),
 ('우리아이', 0.9276459813117981),
 ('어린아이', 0.9251127243041992),
 ('외동아이', 0.923448383808136),
 ('큰아이', 0.9182673692703247),
 ('아이템이', 0.9045064449310303),
 ('아이돌보미가', 0.895254373550415),
 ('아이디어가', 0.8829282522201538)]

In [19]:
most_similar("여아")

[('영아', 0.8956602215766907),
 ('역아', 0.8942204117774963),
 ('미숙아', 0.8842177987098694),
 ('소아', 0.8608177304267883),
 ('성빈아', 0.860334038734436),
 ('모아', 0.850426197052002),
 ('조산아', 0.8490022420883179),
 ('이케아', 0.8407268524169922),
 ('네오포비아', 0.8357229232788086),
 ('복숭아', 0.832750141620636)]

In [20]:
most_similar("남자아이")

[('여자아이', 0.9715147614479065),
 ('큰아이', 0.9432119131088257),
 ('외동아이', 0.9336391687393188),
 ('경희아이', 0.9319890737533569),
 ('아이돌이', 0.918034017086029),
 ('우리아이', 0.9120261073112488),
 ('어린아이', 0.9109992384910583),
 ('작은아이', 0.9014006853103638),
 ('아이템이', 0.8865754008293152),
 ('남자아이가', 0.8698145151138306)]

In [21]:
most_similar("남아")

[('참아', 0.8923928737640381),
 ('삼아', 0.8810754418373108),
 ('짧아', 0.8605631589889526),
 ('낳아', 0.8422902226448059),
 ('낮아', 0.8398402333259583),
 ('쌓아', 0.8363903164863586),
 ('동남아', 0.8339701890945435),
 ('귀담아', 0.8317490220069885),
 ('닮아', 0.8304253816604614),
 ('닿아', 0.8183083534240723)]

In [22]:
most_similar("치아")

[('무뇌아', 0.8737269639968872),
 ('치아관리', 0.841048002243042),
 ('소아', 0.8387740850448608),
 ('코코아', 0.8343782424926758),
 ('박테리아', 0.8338569402694702),
 ('모아', 0.8238998651504517),
 ('성빈아', 0.8150904774665833),
 ('단태아', 0.8110718131065369),
 ('아토피', 0.8068982362747192),
 ('아토팜', 0.7934300899505615)]

In [23]:
most_similar("이빨")

[('이어갈', 0.9305525422096252),
 ('이날', 0.9090676307678223),
 ('이겨낼', 0.8946222066879272),
 ('이달', 0.8883301615715027),
 ('이길', 0.8756088614463806),
 ('이룰', 0.8683262467384338),
 ('이행할', 0.8568215370178223),
 ('이동할', 0.8438403606414795),
 ('이쯤', 0.8256165981292725),
 ('이걸', 0.8225879669189453)]

In [24]:
most_similar("놀이")

[('공놀이', 0.9686663746833801),
 ('블록놀이', 0.9656417965888977),
 ('까꿍놀이', 0.9508370757102966),
 ('음악놀이', 0.9466544389724731),
 ('공구놀이', 0.946345329284668),
 ('모래놀이', 0.941689133644104),
 ('촉감놀이', 0.9382611513137817),
 ('소꿉놀이', 0.9382520914077759),
 ('오감놀이', 0.9297782778739929),
 ('놀이터가', 0.929618239402771)]

In [25]:
most_similar("감기")

[('열감기', 0.9675351977348328),
 ('코감기', 0.9626106023788452),
 ('증기', 0.9424244165420532),
 ('환절기', 0.9331618547439575),
 ('장기', 0.9271147847175598),
 ('쉽기', 0.9224294424057007),
 ('흡입기', 0.920589029788971),
 ('황기', 0.9200927019119263),
 ('변기', 0.9173961281776428),
 ('감염되기', 0.9128080606460571)]

### 결과 확인
하이퍼 파라미터 조정을 다양하게 시도해보며 결과를 확인해봤지만, 역시 사용하기 어렵겠다는 결론을 내렸다. 말뭉치가 부족한 것이 원인이 아닐까?